In [46]:
import pandas as pd
from geopy.geocoders import Nominatim
import time


#pd.set_option('display.max_columns', None)  #show all columns when displaying dataframe
#pd.set_option('display.max_rows', None)  #show all rows when displaying dataframe

tornado_df = pd.read_csv('../Datasets/Tornado_Tracks.csv') #Use relative path to your csv file
#df = df.head()  # first look at dataset

pd.set_option('display.max_columns', None)

tornado_df.head()



,State Abbreviation,State FIPS,Date,Year,Month,Day,"Date (Year, Month, Day)",Time,Time Zone,Magnitude,Width (yards),Length (miles),Fatalities,Injuries,Crop Loss,Property Loss,Tornado Number,State Number,EF Scale (unaltered or previous rating),Starting Latitude,Starting Longitude,Ending Latitude,Ending Longitude,Object ID,Shape Length
0,OK,40,10/1/1950 5:00:00 AM,1950,10,1,1950-10-01,21:00:00,3,1,10,15.8,0,0,0.0,4.0,192,23,0,36.73,-102.52,36.8800,-102.3000,1,32166.649032
1,NC,37,10/9/1950 5:00:00 AM,1950,10,9,1950-10-09,02:15:00,3,3,880,2.0,0,3,0.0,5.0,193,9,0,34.17,-78.60,34.1701,-78.5999,2,17.462751
2,KY,21,11/20/1950 5:00:00 AM,1950,11,20,1950-11-20,02:20:00,3,2,10,0.1,0,0,0.0,5.0,195,1,0,37.37,-87.20,37.3701,-87.1999,3,17.891905
3,KY,21,11/20/1950 5:00:00 AM,1950,11,20,1950-11-20,04:00:00,3,1,10,0.1,0,0,0.0,5.0,196,2,0,38.20,-84.50,38.2001,-84.4999,4,18.016042
4,MS,28,11/20/1950 5:00:00 AM,1950,11,20,1950-11-20,07:30:00,3,1,37,2.0,0,3,0.0,4.0,197,14,0,32.42,-89.13,32.4201,-89.1299,5,17.257657


In [47]:
tornado_df.info() #check info of dataframe (data types, non-null values)
tornado_df.shape  #check shape of dataframe (rows, columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71813 entries, 0 to 71812
Data columns (total 25 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   State Abbreviation                       71813 non-null  object 
 1   State FIPS                               71813 non-null  int64  
 2   Date                                     71813 non-null  object 
 3   Year                                     71813 non-null  int64  
 4   Month                                    71813 non-null  int64  
 5   Day                                      71813 non-null  int64  
 6   Date (Year, Month, Day)                  71813 non-null  object 
 7   Time                                     71813 non-null  object 
 8   Time Zone                                71813 non-null  int64  
 9   Magnitude                                71813 non-null  int64  
 10  Width (yards)                            71813

(71813, 25)

In [48]:
tornado_df.isnull().sum()  #check for null values in each column

State Abbreviation                             0
State FIPS                                     0
Date                                           0
Year                                           0
Month                                          0
Day                                            0
Date (Year, Month, Day)                        0
Time                                           0
Time Zone                                      0
Magnitude                                      0
Width (yards)                                  0
Length (miles)                                 0
Fatalities                                     0
Injuries                                       0
Crop Loss                                      0
Property Loss                                  0
Tornado Number                                 0
State Number                                   0
EF Scale (unaltered or previous rating)        0
Starting Latitude                              0
Starting Longitude  

In [49]:
tornado_df.isna()  #to see any nulls in the dataset 

tornado_df.isna().sum() #to get count by column for any nulls

State Abbreviation                             0
State FIPS                                     0
Date                                           0
Year                                           0
Month                                          0
Day                                            0
Date (Year, Month, Day)                        0
Time                                           0
Time Zone                                      0
Magnitude                                      0
Width (yards)                                  0
Length (miles)                                 0
Fatalities                                     0
Injuries                                       0
Crop Loss                                      0
Property Loss                                  0
Tornado Number                                 0
State Number                                   0
EF Scale (unaltered or previous rating)        0
Starting Latitude                              0
Starting Longitude  

In [50]:
tornado_df['EF Scale (unaltered or previous rating)'].unique()  #trying to determine if this column is needed 
tornado_df['EF Scale (unaltered or previous rating)'].value_counts()



EF Scale (unaltered or previous rating)
0    69950
1     1863
Name: count, dtype: int64

In [51]:
tornado_df['Object ID'].unique()  #trying to determine if this column is needed 
#tornado_df['EF Scale (unaltered or previous rating)'].value_counts()

array([    1,     2,     3, ..., 71811, 71812, 71813], shape=(71813,))

#### Drop columns

DROP
- Date, Year, Month, Day : redundant because there is a column with all of this information - keeping year and month  
- Object ID - appears to be just a numbered listing 
- Fatalities - i only care about if the number of tornados have increased and are moving eastward to create a new tornado alley
- Injuries - same as fatalities
- Property Damage - not relevant to my analysis

KEEP
- State FIPS is a unique identifier for each state, will keep for now.. 
- EF Scale - debated on getting rid of because of the unique counts, but will keep for now
- Shape Length is how far the tornado traveled, will keep -- do I need longitude and latitude if I have this?  Yes if I want to plot on a US Graph later.  



In [52]:
#Drop/Rename Columns
tornado_df.columns


Index(['State Abbreviation', 'State FIPS', 'Date', 'Year', 'Month', 'Day',
       'Date (Year, Month, Day)', 'Time', 'Time Zone', 'Magnitude',
       'Width (yards)', 'Length (miles)', 'Fatalities', 'Injuries',
       'Crop Loss', 'Property Loss', 'Tornado Number', 'State Number',
       'EF Scale (unaltered or previous rating)', 'Starting Latitude',
       'Starting Longitude', 'Ending Latitude', 'Ending Longitude',
       'Object ID', 'Shape Length'],
      dtype='object')

In [53]:
tornado_df.drop(columns=['Date','Day','Fatalities','Injuries','Crop Loss', 'Property Loss','Object ID'], inplace=True)

In [54]:
tornado_df #making sure columns dropped


,State Abbreviation,State FIPS,Year,Month,"Date (Year, Month, Day)",Time,Time Zone,Magnitude,Width (yards),Length (miles),Tornado Number,State Number,EF Scale (unaltered or previous rating),Starting Latitude,Starting Longitude,Ending Latitude,Ending Longitude,Shape Length
0,OK,40,1950,10,1950-10-01,21:00:00,3,1,10,15.80,192,23,0,36.7300,-102.5200,36.8800,-102.3000,32166.649032
1,NC,37,1950,10,1950-10-09,02:15:00,3,3,880,2.00,193,9,0,34.1700,-78.6000,34.1701,-78.5999,17.462751
2,KY,21,1950,11,1950-11-20,02:20:00,3,2,10,0.10,195,1,0,37.3700,-87.2000,37.3701,-87.1999,17.891905
3,KY,21,1950,11,1950-11-20,04:00:00,3,1,10,0.10,196,2,0,38.2000,-84.5000,38.2001,-84.4999,18.016042
4,MS,28,1950,11,1950-11-20,07:30:00,3,1,37,2.00,197,14,0,32.4200,-89.1300,32.4201,-89.1299,17.257657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71808,IN,18,2024,9,2024-09-07,05:58:00,3,-9,10,0.05,624887,0,0,41.7019,-86.9567,41.7012,-86.9562,118.282700
71809,WY,56,2024,9,2024-09-09,14:19:00,3,0,25,0.19,624888,0,0,43.6457,-108.2136,43.6466,-108.2100,423.993166
71810,NY,36,2024,9,2024-09-09,14:33:00,3,0,75,1.20,624889,0,0,43.0333,-79.0102,43.0201,-78.9972,2476.801795
71811,NY,36,2024,9,2024-09-09,15:15:00,3,-9,50,0.10,624890,0,0,43.7565,-75.8837,43.7551,-75.8815,326.393737


In [55]:
#Check column data types
tornado_df.dtypes

State Abbreviation                          object
State FIPS                                   int64
Year                                         int64
Month                                        int64
Date (Year, Month, Day)                     object
Time                                        object
Time Zone                                    int64
Magnitude                                    int64
Width (yards)                                int64
Length (miles)                             float64
Tornado Number                               int64
State Number                                 int64
EF Scale (unaltered or previous rating)      int64
Starting Latitude                          float64
Starting Longitude                         float64
Ending Latitude                            float64
Ending Longitude                           float64
Shape Length                               float64
dtype: object

In [56]:
tornado_df.loc[tornado_df.duplicated(keep=False)] # checking for duplicates

,State Abbreviation,State FIPS,Year,Month,"Date (Year, Month, Day)",Time,Time Zone,Magnitude,Width (yards),Length (miles),Tornado Number,State Number,EF Scale (unaltered or previous rating),Starting Latitude,Starting Longitude,Ending Latitude,Ending Longitude,Shape Length
59808,IA,19,2015,6,2015-06-20,18:32:00,3,1,50,2.92,610626,0,0,40.8123,-91.2844,40.7783,-91.2494,6338.39183
59809,IA,19,2015,6,2015-06-20,18:32:00,3,1,50,2.92,610626,0,0,40.8123,-91.2844,40.7783,-91.2494,6338.39183


In [57]:
tornado_df.drop_duplicates(keep= "first", inplace=True)  #drops duplicates keeping the first instance

In [58]:
# check again for duplicates after dropping

tornado_df.loc[tornado_df.duplicated(keep=False)] 

,State Abbreviation,State FIPS,Year,Month,"Date (Year, Month, Day)",Time,Time Zone,Magnitude,Width (yards),Length (miles),Tornado Number,State Number,EF Scale (unaltered or previous rating),Starting Latitude,Starting Longitude,Ending Latitude,Ending Longitude,Shape Length


In [59]:
tornado_df.rename({'State Abbreviation': 'State'}, axis=1, inplace=True)
tornado_df.head()

,State,State FIPS,Year,Month,"Date (Year, Month, Day)",Time,Time Zone,Magnitude,Width (yards),Length (miles),Tornado Number,State Number,EF Scale (unaltered or previous rating),Starting Latitude,Starting Longitude,Ending Latitude,Ending Longitude,Shape Length
0,OK,40,1950,10,1950-10-01,21:00:00,3,1,10,15.8,192,23,0,36.73,-102.52,36.8800,-102.3000,32166.649032
1,NC,37,1950,10,1950-10-09,02:15:00,3,3,880,2.0,193,9,0,34.17,-78.60,34.1701,-78.5999,17.462751
2,KY,21,1950,11,1950-11-20,02:20:00,3,2,10,0.1,195,1,0,37.37,-87.20,37.3701,-87.1999,17.891905
3,KY,21,1950,11,1950-11-20,04:00:00,3,1,10,0.1,196,2,0,38.20,-84.50,38.2001,-84.4999,18.016042
4,MS,28,1950,11,1950-11-20,07:30:00,3,1,37,2.0,197,14,0,32.42,-89.13,32.4201,-89.1299,17.257657


In [60]:
# Filter to only the five states listed

states = ["KY", "KS", "OK", "MO", "AR"]
filtered_df = tornado_df[
    (tornado_df["State"].isin(states)) &
    (tornado_df["Year"] >= 2005)
    ]

filtered_df



,State,State FIPS,Year,Month,"Date (Year, Month, Day)",Time,Time Zone,Magnitude,Width (yards),Length (miles),Tornado Number,State Number,EF Scale (unaltered or previous rating),Starting Latitude,Starting Longitude,Ending Latitude,Ending Longitude,Shape Length
46934,KS,20,2005,10,2005-10-19,15:36:00,3,0,30,0.30,1196,116,0,37.3800,-100.4500,37.3800,-100.4500,NaN
46935,KS,20,2005,10,2005-10-19,15:42:00,3,0,30,0.20,1197,117,0,37.5200,-100.3000,37.5200,-100.3000,NaN
46936,OK,40,2005,10,2005-10-19,16:47:00,3,1,400,13.00,1195,27,0,36.6700,-99.4200,36.7200,-99.2000,25455.061195
46960,MO,29,2005,11,2005-11-12,19:40:00,3,1,75,5.00,535,27,0,37.0800,-93.8300,37.0500,-93.7700,7882.077783
46961,AR,5,2005,11,2005-11-12,22:27:00,3,0,30,3.50,599,25,0,34.5700,-94.3200,34.5700,-94.2500,7792.364300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71652,KY,21,2024,7,2024-07-09,14:40:00,3,1,150,6.33,624666,0,0,37.6830,-87.9750,37.7710,-87.9430,12887.578856
71662,KS,20,2024,8,2024-08-14,18:25:00,3,-9,20,0.48,624855,0,0,39.5330,-96.5430,39.5320,-96.5340,1012.218747
71663,KS,20,2024,8,2024-08-14,18:39:00,3,-9,15,0.01,624856,0,0,39.5060,-96.4410,39.5060,-96.4410,NaN
71689,MO,29,2024,8,2024-08-31,13:44:00,3,-9,20,0.09,624882,0,0,36.7819,-90.3494,36.7827,-90.3507,182.499866


In [62]:

## used copilot to convert lat/lon to city name


# Initialize geocoder
geolocator = Nominatim(user_agent="geoapi")

# Define reverse geocoding function
def get_city(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), timeout=10)
        return location.raw['address'].get('city', '') or \
               location.raw['address'].get('town', '') or \
               location.raw['address'].get('village', '')
    except:
        return None

# Apply to each row using Starting Latitude and Longitude
filtered_df["Starting City"] = filtered_df.apply(
    lambda row: get_city(row["Starting Latitude"], row["Starting Longitude"]),
    axis=1
)

# Optional: pause between requests to avoid rate limits
time.sleep(1)



C:\Users\15025\AppData\Local\Temp\ipykernel_12300\1702121150.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["Starting City"] = filtered_df.apply(


In [ ]:
tornado_counts = filtered_df.groupby(['State', 'Year']).size().reset_index(name='tornado_counts')
#average_per_state = tornado_counts.groupby('State')['tornado_counts'].mean()

#average_per_state

tornado_counts

,State,Year,tornado_counts
0,AR,2005,69
1,AR,2006,23
2,AR,2007,15
3,AR,2008,80
4,AR,2009,42
...,...,...,...
95,OK,2020,38
96,OK,2021,63
97,OK,2022,54
98,OK,2023,75


#### Pivot table

Created a pivot table to organize the data in an easier to view way 

In [ ]:
pivot_table = tornado_counts.pivot(index='State', columns='Year', values='tornado_counts').fillna(0)
pivot_table

Year,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
State,,,,,,,,,,,,,,,,,,,,
AR,69,23,15,80,42,31,72,18,31,20,20,23,37,34,29,42,32,40,29,47
KS,136,90,140,185,103,88,67,93,55,40,126,102,60,45,87,17,37,54,45,71
KY,17,19,30,35,32,20,57,41,35,26,17,24,25,35,25,22,36,32,34,48
MO,39,96,42,91,44,62,77,29,48,46,47,18,81,46,63,19,46,14,21,93
OK,27,27,49,77,33,102,118,63,82,16,108,57,85,43,148,38,63,54,75,151
